In [35]:
!pip install transformers

In [3]:
!pip install tensorflow==2.4.1

     |████████████████████████████████| 394.3MB 44kB/s 
     |████████████████████████████████| 10.6MB 32.6MB/s 
     |████████████████████████████████| 471kB 42.2MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [2]:
!pip install tensorflow_addons

In [ ]:
!pip install kaggle

In [62]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c jigsaw-multilingual-toxic-comment-classification

  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 43.9MB/s]
 94% 28.0M/29.8M [00:00<00:00, 48.7MB/s]
100% 29.8M/29.8M [00:00<00:00, 50.8MB/s]
  0% 0.00/612k [00:00<?, ?B/s]
100% 612k/612k [00:00<00:00, 83.2MB/s]
 97% 12.0M/12.4M [00:00<00:00, 39.6MB/s]
100% 12.4M/12.4M [00:00<00:00, 41.4MB/s]
  0% 0.00/3.44M [00:00<?, ?B/s]
100% 3.44M/3.44M [00:00<00:00, 56.7MB/s]
 95% 278M/292M [00:02<00:00, 174MB/s]
100% 292M/292M [00:02<00:00, 131MB/s]
 88% 70.0M/79.6M [00:00<00:00, 78.0MB/s]
100% 79.6M/79.6M [00:01<00:00, 82.9MB/s]
100% 647M/650M [00:05<00:00, 84.2MB/s]
100% 650M/650M [00:05<00:00, 114MB/s] 
 97% 36.0M/37.3M [00:00<00:00, 46.3MB/s]
100% 37.3M/37.3M [00:00<00:00, 45.3MB/s]


In [169]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM,Bidirectional,Input,Dense,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.models import Sequential,Model,save_model,load_model
import transformers
from tqdm import tqdm
import re
from tokenizers import BertWordPieceTokenizer
from transformers import AdamW,get_linear_schedule_with_warmup, BertModel, BertConfig, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow_addons.optimizers import AdamW
from tensorflow.keras import backend as K

In [37]:
tf.__version__

'2.4.1'

In [38]:
K.clear_session()

In [5]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit = 1e10

The Jupyter HTML Notebook.

This launches a Tornado based HTML Notebook Server that serves up an
HTML5/Javascript Notebook client.

Subcommands
-----------

Subcommands are launched as `jupyter-notebook cmd [args]`. For information on
using subcommand 'cmd', do: `jupyter-notebook cmd -h`.

stop
    Stop currently running notebook server for a given port
password
    Set a password for the notebook server.
list
    List currently running notebook servers.

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--script
    DEPRECATED, IGNORED
--pylab
    DISABLED: use %pylab or %matplotlib in the notebook to enable matplotlib.
--debug
    set log level to logging.DEBUG (maximize logging output)
--no-browser
    Don't open the notebook in a browser after startup.
--allow-root
    Allow the notebook to be run from root user.
-y
    Answer y

In [7]:
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel

ERROR: Operation cancelled by user


In [ ]:
from google.colab import drive
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [39]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.113.11.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.113.11.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.113.11.186:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
train_data = pd.read_csv('jigsaw-toxic-comment-train.csv.zip',compression='zip', header=0, sep=',', quotechar='"')
#val_data = pd.read_csv('validation.csv.zip',compression='zip', header=0, sep=',', quotechar='"')
#test_data = pd.read_csv('test.csv.zip',compression='zip', header=0, sep=',', quotechar='"')

In [ ]:
train_2 = pd.read_csv('jigsaw-unintended-bias-train.csv.zip',compression='zip', header=0, sep=',', quotechar='"')
train_2.toxic = train_2.toxic.round().astype(int)

In [ ]:
train_data = pd.concat([train_data[['comment_text','toxic']],
                        train_2[['comment_text','toxic']].query('toxic == 1'),
                        train_2[['comment_text','toxic']].query('toxic == 0').sample(n = 20000, random_state = 0)
                      ])

In [ ]:
train_data['toxic'].value_counts()

0    222165
1    133610
Name: toxic, dtype: int64

In [ ]:
y_train = train_data.toxic.values
#y_val = val_data.toxic.values

In [ ]:
from sklearn.model_selection import train_test_split
train_data,val_data = train_test_split(train_data,test_size = 0.1, random_state = 42)

In [ ]:
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320197 entries, 12147 to 121958
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_text  320197 non-null  object
 1   toxic         320197 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.3+ MB
None


In [40]:
#loading other dataset

df1 = pd.read_csv('/content/gdrive/MyDrive/twitter_sentiment_analysis/Twitter_Data.csv')
print(df1['category'].value_counts())
df2 = pd.read_csv('/content/gdrive/MyDrive/twitter_sentiment_analysis/Reddit_Data.csv')
print(df2['category'].value_counts())
df = pd.concat([df1, df2])
print(df['category'].value_counts())

df.drop(columns=['clean_comment'], axis = 1, inplace = True)
df.dropna(inplace = True)
print(df['category'].value_counts())

df['category'] = df['category'].apply(lambda x: 2 if x == -1 else x)
print(df.head(5))
print(df['category'].value_counts())

 1.0    72250
 0.0    55213
-1.0    35510
Name: category, dtype: int64
 1    15830
 0    13142
-1     8277
Name: category, dtype: int64
 1.0    88080
 0.0    68355
-1.0    43787
Name: category, dtype: int64
 1.0    72249
 0.0    55211
-1.0    35509
Name: category, dtype: int64
                                          clean_text  category
0  when modi promised “minimum government maximum...       2.0
1  talk all the nonsense and continue all the dra...       0.0
2  what did just say vote for modi  welcome bjp t...       1.0
3  asking his supporters prefix chowkidar their n...       1.0
4  answer who among these the most powerful world...       1.0
1.0    72249
0.0    55211
2.0    35509
Name: category, dtype: int64


In [41]:
train_x, val_x, train_y, val_y = train_test_split(df['clean_text'], df['category'], test_size = 0.2, random_state = 42)
print(len(train_x))
print(len(val_x))

130375
32594


In [42]:
print(train_x[:5])
print(train_y[:5])

63425     some media guys cant even think straight forwa...
78455                                             only namo
70129     does this add the space debris problem the awa...
157073    today’ interaction modi phenomenal looking fre...
120885    more modi bjp wants change nitin gadkari will ...
Name: clean_text, dtype: object
63425     1.0
78455     0.0
70129     1.0
157073    1.0
120885    1.0
Name: category, dtype: float64


In [43]:
AUTO = tf.data.experimental.AUTOTUNE
max_len = 256
BATCH_SIZE = 16*strategy.num_replicas_in_sync
EPOCHS = 20

In [45]:
from tokenizers import BertWordPieceTokenizer

tokenizer = transformers.DistilBertTokenizer.from_pretrained('bert-base-uncased',lowercase = True)
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [46]:
def convert_example_to_feature(sentence):
  return tokenizer.encode_plus(
      sentence,
      add_special_tokens = True,
      max_length = max_len,
      padding = True,
      truncation = True,
      return_attention_mask = True,
      return_token_type_ids = True
  )

In [47]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

In [48]:
def encode_example(train_x, train_y):
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []


  for sentence,label in tqdm(zip(train_x.astype(str),train_y.astype(np.int64))):
    bert_input = convert_example_to_feature(sentence)
 
    #     bert_input['token_type_ids'] = bert_input['token_type_ids'] + ([0] * (max_len - len(bert_input['token_type_ids'])))
    #     bert_input['attention_mask'] = bert_input['attention_mask'] + ([0] * (max_len - len(bert_input['attention_mask'])))
  
      
    if(len(bert_input['input_ids']) > max_len):
      for i in range(len(bert_input['input_ids']) - max_len):
        del bert_input['input_ids'][-1]
    else:
      bert_input['input_ids'] = bert_input['input_ids'] + [0]*(max_len - len(bert_input['input_ids']))
    
    if(len(bert_input['token_type_ids']) > max_len):
      for i in range(len(bert_input['token_type_ids']) - max_len):
        del bert_input['token_type_ids'][-1]
    else:
      bert_input['token_type_ids'] = bert_input['token_type_ids'] + [0]*(max_len - len(bert_input['token_type_ids']))

    if(len(bert_input['attention_mask']) > max_len):
      for i in range(len(bert_input['attention_mask']) - max_len):
        del bert_input['attention_mask'][-1]
    else:
      bert_input['attention_mask'] = bert_input['attention_mask'] + [0]*(max_len - len(bert_input['attention_mask']))

    if(len(bert_input['attention_mask']) < max_len or len(bert_input['attention_mask']) > max_len or len(bert_input['token_type_ids']) < max_len or len(bert_input['token_type_ids']) > max_len or len(bert_input['input_ids']) != max_len):
        print('bad')

    input_ids_list.append(bert_input['input_ids'])
    # print(input_ids_list)
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict) 

In [49]:
train_data_encode = encode_example(train_x, train_y).repeat().shuffle(32768).batch(BATCH_SIZE).prefetch(AUTO)
print('done')
#print(train_data_encode)

130375it [01:39, 1309.55it/s]


done


In [50]:
val_data_encode = encode_example(val_x, val_y).batch(BATCH_SIZE).cache().prefetch(AUTO)

32594it [00:23, 1399.89it/s]


In [51]:
print(val_data_encode)

<PrefetchDataset shapes: ({input_ids: (None, 256), token_type_ids: (None, 256), attention_mask: (None, 256)}, (None, 1)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>


In [54]:
#fine tuning of the model.

from transformers import TFBertForSequenceClassification, BertConfig

def build_model():
  config = BertConfig.from_pretrained('bert-base-uncased', num_labels=3)
  model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config = config)

  #model.summary()

  # choosing Adam optimizer
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(2e-5, 2, 0.96)
  wd_schedule = tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 2, 0.9)
  optimizer = AdamW(learning_rate=lr_schedule, weight_decay=lambda : None)
  optimizer.weight_decay = lambda : wd_schedule(optimizer.iterations)
  # optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

  # total_steps = int((len(train_data) / BATCH_SIZE)*EPOCHS)
  # warm_up_steps = int((len(train_data) / BATCH_SIZE)*5) 
  # scheduler = linear_schedule_with_warmup(
  #     optimizer,
  #     num_warmup_steps = warm_up_steps,
  #     num_training_steps = total_steps
  # )

  # we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
  return model

with strategy.scope():
  model = build_model()

model.summary()


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [56]:
callbacks = [ReduceLROnPlateau(factor = 0.2,patience = 4),
             #EarlyStopping(patience = 7)
            ]

EPOCHS = 2
e_steps = int(len(train_x) // BATCH_SIZE)
v_steps = int(len(val_x) // BATCH_SIZE)
bert_history = model.fit(train_data_encode , epochs=EPOCHS, validation_data = val_data_encode, 
                         steps_per_epoch = e_steps, validation_steps = v_steps)

Epoch 1/2
1018/1018 [==============================] - 218s 214ms/step - loss: 0.8298 - accuracy: 0.6328 - val_loss: 0.8246 - val_accuracy: 0.6336
Epoch 2/2
1018/1018 [==============================] - 218s 214ms/step - loss: 0.8311 - accuracy: 0.6318 - val_loss: 0.8246 - val_accuracy: 0.6336


In [60]:
model.save_pretrained('gdrive/MyDrive/sentiment_analysis_v1')

In [61]:
drive.flush_and_unmount()

In [ ]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

In [172]:
config = BertConfig.from_json_file('/content/gdrive/MyDrive/sentiment_analysis_v1/config.json')
# config.output_hidden_states = False
model = TFBertModel.from_pretrained('/content/gdrive/MyDrive/sentiment_analysis_v1', config = config)
# model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at /content/gdrive/MyDrive/sentiment_analysis_v1 were not used when initializing TFBertModel: ['dropout_75', 'classifier']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /content/gdrive/MyDrive/sentiment_analysis_v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [66]:
def convert_example_to_feature(sentence):
  return tokenizer.encode_plus(
      sentence,
      add_special_tokens = True,
      max_length = max_len,
      padding = True,
      truncation = True,
      return_attention_mask = True,
      return_token_type_ids = True
  )

In [101]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

In [125]:
import torch

In [157]:
sentence = "You are a fucking crap person"

input_ids_list = []
token_type_ids_list = []
attention_mask_list = []

bert_input = convert_example_to_feature(sentence)

if(len(bert_input['input_ids']) > max_len):
  for i in range(len(bert_input['input_ids']) - max_len):
    del bert_input['input_ids'][-1]
else:
  bert_input['input_ids'] = bert_input['input_ids'] + [0]*(max_len - len(bert_input['input_ids']))

if(len(bert_input['token_type_ids']) > max_len):
  for i in range(len(bert_input['token_type_ids']) - max_len):
    del bert_input['token_type_ids'][-1]
else:
  bert_input['token_type_ids'] = bert_input['token_type_ids'] + [0]*(max_len - len(bert_input['token_type_ids']))

if(len(bert_input['attention_mask']) > max_len):
  for i in range(len(bert_input['attention_mask']) - max_len):
    del bert_input['attention_mask'][-1]
else:
  bert_input['attention_mask'] = bert_input['attention_mask'] + [0]*(max_len - len(bert_input['attention_mask']))

if(len(bert_input['attention_mask']) < max_len or len(bert_input['attention_mask']) > max_len or len(bert_input['token_type_ids']) < max_len or len(bert_input['token_type_ids']) > max_len or len(bert_input['input_ids']) != max_len):
    print('bad')

input_ids_list.append(bert_input['input_ids'])
# print(input_ids_list)
token_type_ids_list.append(bert_input['token_type_ids'])
attention_mask_list.append(bert_input['attention_mask'])

# input_ids_list = np.expand_dims(np.array(input_ids_list), axis = 0)
# attention_mask_list = np.expand_dims(np.array(attention_mask_list), axis = 0)
# token_type_ids_list = np.expand_dims(np.array(token_type_ids_list), axis = 0)
# label = [None]
# ds = tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label))
input_ids_list = torch.LongTensor(input_ids_list)
token_type_ids_list = torch.LongTensor(token_type_ids_list)
attention_mask_list = torch.LongTensor(attention_mask_list)
# print(input_ids_list.size())
# print(attention_mask_list)
# print(token_type_ids_list)

# predict_dict = {"input_ids": input_ids_list,
#                 "token_type_ids": token_type_ids_list,
#                 "attention_mask": attention_mask_list}

In [158]:
output = model(input_ids = input_ids_list, attention_mask = attention_mask_list, token_type_ids = token_type_ids_list)
# output = model(*ds)

In [160]:
sequence_output = output[0]
cls_token = sequence_output[:, 0, :]

In [162]:
cls_token.shape

torch.Size([1, 768])

In [181]:
#training on our twitter sentiment analysis part

def build_model(transformer, max_len=256):
    """
    function for training the BERT model
    """
    input_ids = Input(shape=(max_len,), dtype='int32', name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype='int32', name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype='int32', name="attention_mask")

    sequence_output = transformer(input_ids, token_type_ids, attention_mask)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(3, activation='softmax')(cls_token)
    
    model = Model(inputs=[input_ids, token_type_ids, attention_mask] , outputs=out)
    model.compile(Adam(lr=2e-5), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    
    return model

In [182]:
with strategy.scope():
    config = BertConfig.from_json_file('/content/gdrive/MyDrive/sentiment_analysis_v1/config.json')
    # config.output_hidden_states = False
    transformer_layer = TFBertModel.from_pretrained('/content/gdrive/MyDrive/sentiment_analysis_v1', config = config)
    model = build_model(transformer_layer, max_len=256)
model.summary()

Some layers from the model checkpoint at /content/gdrive/MyDrive/sentiment_analysis_v1 were not used when initializing TFBertModel: ['dropout_75', 'classifier']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /content/gdrive/MyDrive/sentiment_analysis_v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 256)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_14 (TFBertModel)  TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 token_type_ids[0][0]       

In [183]:
callbacks = [ReduceLROnPlateau(factor = 0.2,patience = 3),
             #EarlyStopping(patience = 7)
            ]

EPOCHS = 10
e_steps = int(len(train_x) // BATCH_SIZE)
v_steps = int(len(val_x) // BATCH_SIZE)
bert_history = model.fit(train_data_encode , epochs=EPOCHS, validation_data = val_data_encode, 
                         steps_per_epoch = e_steps, validation_steps = v_steps, callbacks = callbacks)

Epoch 1/10


1018/1018 [==============================] - ETA: 0s - loss: 0.5493 - sparse_categorical_accuracy: 0.7595WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


1018/1018 [==============================] - 300s 225ms/step - loss: 0.5491 - sparse_categorical_accuracy: 0.7596 - val_loss: 0.1157 - val_sparse_categorical_accuracy: 0.9636
Epoch 2/10
1018/1018 [==============================] - 217s 213ms/step - loss: 0.1145 - sparse_categorical_accuracy: 0.9642 - val_loss: 0.0743 - val_sparse_categorical_accuracy: 0.9773
Epoch 3/10
1018/1018 [==============================] - 217s 213ms/step - loss: 0.0709 - sparse_categorical_accuracy: 0.9783 - val_loss: 0.0684 - val_sparse_categorical_accuracy: 0.9785
Epoch 4/10
1018/1018 [==============================] - 217s 214ms/step - loss: 0.0528 - sparse_categorical_accuracy: 0.9831 - val_loss: 0.0595 - val_sparse_categorical_accuracy: 0.9813
Epoch 5/10
1018/1018 [==============================] - 217s 213ms/step - loss: 0.0421 - sparse_categorical_accuracy: 0.9864 - val_loss: 0.0564 - val_sparse_categorical_accuracy: 0.9830
Epoch 6/10
1018/1018 [==============================] - 218s 214ms/step - loss: 0

KeyboardInterrupt: ignored

In [ ]:
#Modi government is dedicated to ‘Jan Kalyan’ but Mamata Didi’s government is busy with ‘Bhatija Kalyan’
#Amazing energy and enthusiasm here at Science City Auditorium, Kolkata. Interacting with our Social Media warriors from all over West Bengal.
#Godi media is not investigated. Planned, voluntary compromise of journalistic ethics isn't investigated... A toolkit is. This is Animal Farm in real life.

In [ ]:
#sentence = "Godi media is not investigated. Planned, voluntary compromise of journalistic ethics isn't investigated... A toolkit is. This is Animal Farm in real life."
sentence = "this is a shit thing"
input_ids_list = []
token_type_ids_list = []
attention_mask_list = []

bert_input = convert_example_to_feature(sentence)

if(len(bert_input['input_ids']) > max_len):
  for i in range(len(bert_input['input_ids']) - max_len):
    del bert_input['input_ids'][-1]
else:
  bert_input['input_ids'] = bert_input['input_ids'] + [0]*(max_len - len(bert_input['input_ids']))

if(len(bert_input['token_type_ids']) > max_len):
  for i in range(len(bert_input['token_type_ids']) - max_len):
    del bert_input['token_type_ids'][-1]
else:
  bert_input['token_type_ids'] = bert_input['token_type_ids'] + [0]*(max_len - len(bert_input['token_type_ids']))

if(len(bert_input['attention_mask']) > max_len):
  for i in range(len(bert_input['attention_mask']) - max_len):
    del bert_input['attention_mask'][-1]
else:
  bert_input['attention_mask'] = bert_input['attention_mask'] + [0]*(max_len - len(bert_input['attention_mask']))

if(len(bert_input['attention_mask']) < max_len or len(bert_input['attention_mask']) > max_len or len(bert_input['token_type_ids']) < max_len or len(bert_input['token_type_ids']) > max_len or len(bert_input['input_ids']) != max_len):
    print('bad')

input_ids_list.append(bert_input['input_ids'])
# print(input_ids_list)
token_type_ids_list.append(bert_input['token_type_ids'])
attention_mask_list.append(bert_input['attention_mask'])

# input_ids_list = np.expand_dims(np.array(input_ids_list), axis = 0)
# attention_mask_list = np.expand_dims(np.array(attention_mask_list), axis = 0)
# token_type_ids_list = np.expand_dims(np.array(token_type_ids_list), axis = 0)
# label = [None]
# ds = tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label))
input_ids_list = tf.constant(input_ids_list)
token_type_ids_list = tf.constant(token_type_ids_list)
attention_mask_list = tf.constant(attention_mask_list)
# print(input_ids_list.size())
# print(attention_mask_list)
# print(token_type_ids_list)

# predict_dict = {"input_ids": input_ids_list,
#                 "token_type_ids": token_type_ids_list,
#                 "attention_mask": attention_mask_list}

In [242]:
output = model.predict([input_ids_list, token_type_ids_list, attention_mask_list])

In [243]:
classes = ['neutral', 'positive', 'negative']

In [244]:
val = np.argmax(output, axis = 1)
print(val)
print(output)

[2]
[[3.5443401e-04 3.1661220e-05 9.9961394e-01]]


In [245]:
print(classes[val[0]])
print(output[0][val[0]])

negative
0.99961394
